# UNIDADE 3:  Regressão Linear

3.1 Previsões simples (Regressão linear)

3.2 Previsões complexas (Regressão linear múltipla)

3.3   Algoritmos de Classificação

3.3.1 Métodos simbólicos: Árvore de Decisão

3.3.2 Métodos ensemble: Random Forest


<div style="border: 2px solid #00008B; padding: 15px; border-radius: 10px; background-color: #00008B; color: #FFFFFF; font-family: Arial;">
  <h1 style="margin-top: 0;"> Predição de Cargas de Trabalho para Juízes</h1>
  <p>Descrição: Prever o volume de trabalho em diferentes tribunais com base em fatores como tamanho da população, tipos de casos comuns na região, etc.
</p>
</div>


<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
  <ul>
    <li>Tamanho da população da região onde o tribunal está localizado (valores real entre 30,000 e 500,000)</li>
    <li>Número de juízes ativos no tribunal (valores inteiros entre 1 e 20)</li>
    <li>Número de casos recebidos mensalmente (valores inteiros entre 150 e 800)</li>
    <li>Distribuição dos tipos de casos comuns na região (valores: criminal, civil, trabalhista)</li>
    <li>Taxa de crescimento da população na região (valores reais entre 0.0 e 1.0)</li>
    <li>Taxa de criminalidade na região (valores reais entre 0.0 e 1.0)</li>
    <li>Média de tempo gasto em cada tipo de caso em meses (número inteiro)</li>
    <li>Nível de automação e eficiência do tribunal (valores inteiros entre 0 e 10)</li>
    <li>Nível de congestionamento do sistema judicial na região (baixo, médio, alto)</li>
    <li>Número de advogados atuando na região (valores inteiros entre 1 e 100)</li>
    <li>Número de prédios judiciais na região (valores inteiros entre 1 e 10)</li>
    <li>Nível de urbanização da região (baixo, médio, alto)</li>
    <li>Nível de educação da população na região (valores reais entre 0.0 e 1.0)</li>
    <li>Número de habitantes por juiz (valor fruto da divisão de tamanho da população da região pelo número de juízes ativos no tribunal)</li>
    <li>Índice de Desenvolvimento Humano (valores reais entre 0.0 e 1.0)</li>
    <li>Média de idade dos  es (entre 35 e 60)</li>
    <li><b>Número de casos pendentes atualmente no tribunal (valores inteiros, coluna que não pode ser nula)</b></li>
  </ul>
</div>




In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', '{:.5f}'.format)

import streamlit as st
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Ingestão de Dados

In [ ]:
df_servidor = pd.read_csv("../data/raw/predicao_cargas_trabalho.csv")

In [ ]:
df_servidor.head(5)

# Limpeza e transformação de dados

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: codigo_tribunal</h3>
</div>

In [ ]:
df_servidor["codigo_tribunal"].unique()

In [ ]:
df_servidor = df_servidor.dropna(subset=['codigo_tribunal'])
df_servidor["codigo_tribunal"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: populacao</h3>
</div>

In [ ]:
df_servidor["populacao"].unique()
df_servidor['populacao'].fillna(df_servidor['populacao'].mean(), inplace=True)
df_servidor["populacao"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: num_juizes</h3>
</div>

In [ ]:
df_servidor["num_juizes"].unique()
df_servidor = df_servidor.dropna(subset=['num_juizes'])
df_servidor["num_juizes"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: casos_recebidos</h3>
</div>

In [ ]:
df_servidor["casos_recebidos"].value_counts(dropna=False)
df_servidor['casos_recebidos'].fillna(df_servidor['casos_recebidos'].mean(), inplace=True)
df_servidor["casos_recebidos"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: tipo_caso</h3>
</div>

In [ ]:
df_servidor["tipo_caso"].unique()
df_servidor['tipo_caso'].fillna('civil', inplace=True)
df_servidor["tipo_caso"].value_counts(dropna=False)

In [ ]:
categorias = df_servidor['tipo_caso'].astype('category').cat.categories
codigos = df_servidor['tipo_caso'].astype('category').cat.codes
print({codigo: categoria for codigo, categoria in enumerate(categorias)})

In [ ]:
df_servidor['tipo_caso'] = df_servidor['tipo_caso'].astype('category').cat.codes
df_servidor["tipo_caso"].unique()

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: taxa_crescimento_populacao</h3>
</div>

In [ ]:
df_servidor["taxa_crescimento_populacao"].value_counts(dropna=False)

In [ ]:
df_servidor = df_servidor.dropna(subset=['taxa_crescimento_populacao'])
df_servidor["taxa_crescimento_populacao"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: tempo_casos</h3>
</div>

In [ ]:
df_servidor["tempo_casos"].unique()

In [ ]:
df_servidor = df_servidor.dropna(subset=['tempo_casos'])
df_servidor["tempo_casos"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: nivel_automacao</h3>
</div>

In [ ]:
df_servidor["nivel_automacao"].unique()

In [ ]:
from sklearn.impute import SimpleImputer

# Imputação com o mais frequente
imputer = SimpleImputer(strategy='most_frequent')
df_servidor['nivel_automacao'] = imputer.fit_transform(df_servidor[['nivel_automacao']])

In [ ]:
#df_servidor['nivel_automacao'].fillna(5, inplace=True)

In [ ]:
df_servidor["nivel_automacao"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: congestionamento</h3>
</div>

In [ ]:
df_servidor["congestionamento"].unique()

In [ ]:
df_servidor['congestionamento'].fillna('invalido', inplace=True)
df_servidor["congestionamento"].value_counts(dropna=False)

In [ ]:
categorias = df_servidor['congestionamento'].astype('category').cat.categories
codigos = df_servidor['congestionamento'].astype('category').cat.codes
print({codigo: categoria for codigo, categoria in enumerate(categorias)})

In [ ]:
df_servidor['congestionamento'] = df_servidor['congestionamento'].astype('category').cat.codes
df_servidor["congestionamento"].unique()

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: advogados</h3>
</div>

In [ ]:
df_servidor["advogados"].unique()

In [ ]:
df_servidor = df_servidor.dropna(subset=['advogados'])
df_servidor["advogados"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: urbanizacao</h3>
</div>

In [ ]:
df_servidor["urbanizacao"].unique()

In [ ]:
df_servidor['urbanizacao'].fillna('desconhecido', inplace=True)
df_servidor["urbanizacao"].value_counts(dropna=False)

In [ ]:
categorias = df_servidor['urbanizacao'].astype('category').cat.categories
codigos = df_servidor['urbanizacao'].astype('category').cat.codes
print({codigo: categoria for codigo, categoria in enumerate(categorias)})

In [ ]:
df_servidor['urbanizacao'] = df_servidor['urbanizacao'].astype('category').cat.codes
df_servidor["urbanizacao"].unique()

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: nivel_educacao</h3>
</div>

In [ ]:
df_servidor["nivel_educacao"].unique()

In [ ]:
df_servidor["nivel_educacao"].value_counts(dropna=False)
df_servidor['nivel_educacao'].fillna(df_servidor['nivel_educacao'].mean(), inplace=True)
df_servidor["nivel_educacao"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: habitantes_por_juiz</h3>
</div>

In [ ]:
df_servidor["habitantes_por_juiz"].unique()

In [ ]:
df_servidor = df_servidor.dropna(subset=['habitantes_por_juiz'])
df_servidor["habitantes_por_juiz"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: idh</h3>
</div>

In [ ]:
df_servidor["idh"].unique()

In [ ]:
df_servidor["idh"].value_counts(dropna=False)
df_servidor['idh'].fillna(df_servidor['idh'].mean(), inplace=True)
df_servidor["idh"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: idade_juizes</h3>
</div>

In [ ]:
df_servidor["idade_juizes"].unique()

In [ ]:
df_servidor["idade_juizes"].value_counts(dropna=False)
df_servidor['idade_juizes'].fillna(df_servidor['idade_juizes'].mean(), inplace=True)
df_servidor["idade_juizes"].value_counts(dropna=False)

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: casos_pendentes</h3>
</div>

In [ ]:
df_servidor["casos_pendentes"].unique()

In [ ]:
df_servidor = df_servidor.dropna(subset=['casos_pendentes'])
df_servidor["casos_pendentes"].value_counts(dropna=False)

In [ ]:
df_servidor.isna().sum()

<div style="background-color: #f0f8ff; padding: 7px; border-radius: 10px;">
<h3>Variável: idade_juizes</h3>
</div>

In [ ]:
bins = [18, 30, 40, 50, 60, 75, 80, 85]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']
df_servidor['idade_juizes'] = pd.cut(df_servidor['idade_juizes'], bins=bins, labels=labels, right=True, include_lowest=True)
df_servidor["idade_juizes"].unique()

In [ ]:
df_servidor["idade_juizes"].value_counts(dropna=False)

In [ ]:
df_servidor = pd.get_dummies(df_servidor, columns=['idade_juizes'], prefix='idade_juizes')
dummy_columns = [col for col in df_servidor.columns if col.startswith('idade_juizes_')]
df_servidor[dummy_columns] = df_servidor[dummy_columns].astype(int)
df_servidor

# Ajustes de tipos

In [ ]:
df_servidor['casos_pendentes'] = df_servidor['casos_pendentes'].astype(int)
df_servidor['casos_recebidos'] = df_servidor['casos_recebidos'].astype(int)

In [ ]:
df_servidor.drop(columns=['codigo_tribunal'], inplace=True)

In [ ]:
df_servidor.isna().sum()

In [ ]:
df_servidor.shape

In [ ]:
df_servidor.head(5)

In [ ]:
# Salvar o DataFrame como um arquivo CSV
df_servidor.to_csv('../data/interim/predicao_cargas_trabalho_processado.csv', index=False)

# Normalização e Padronização dos dados

### Padronização (Z-score Scaling)

$z = \frac{x - \mu}{\sigma}$


In [ ]:
scaler = StandardScaler()
df_normalizado = pd.DataFrame(scaler.fit_transform(df_servidor), columns=df_servidor.columns)
df_normalizado

### Normalização (Min-Max Scaling)

$z = \frac{x - \min(x)}{\max(x) - \min(x)}$

In [ ]:
scaler = MinMaxScaler()
df_normalizado = pd.DataFrame(scaler.fit_transform(df_servidor), columns=df_servidor.columns)
df_normalizado

In [ ]:
# Salvar o scaler ajustado
joblib.dump(scaler, '../models/scaler.pkl')

In [ ]:
# Salvar o DataFrame como um arquivo CSV
df_normalizado.to_csv('../data/processed/predicao_cargas_trabalho_normalizado.csv', index=False)